In [2]:
import pandas as pd
import numpy as np
from pandas import DataFrame as df 
import math
import random as rand


from sklearn.metrics import silhouette_score,calinski_harabasz_score,davies_bouldin_score
from sklearn.cluster import KMeans
from sklearn import metrics         
import copy 
from tabulate import tabulate
import random

In [3]:
# data=pd.read_csv("total_data_na.csv")
data=pd.read_csv("../Dataset/school_district_breakdowns_1.csv")
data=data.dropna()
data=data.drop("jurisdiction_name",axis=1)
data

,count_participants,count_female,percent_female,count_male,percent_male,count_gender_unknown,percent_gender_unknown,count_gender_total,percent_gender_total,count_pacific_islander,...,count_citizen_status_total,percent_citizen_status_total,count_receives_public_assistance,percent_receives_public_assistance,count_nreceives_public_assistance,percent_nreceives_public_assistance,count_public_assistance_unknown,percent_public_assistance_unknown,count_public_assistance_total,percent_public_assistance_total
0,11,4,0.36,7,0.64,False,False,11,100,False,...,11,100,0,0.00,11,1.00,False,False,11,100
1,124,70,0.56,54,0.44,False,False,124,100,False,...,124,100,38,0.31,86,0.69,False,False,124,100
2,38,26,0.68,12,0.32,False,False,38,100,False,...,38,100,14,0.37,24,0.63,False,False,38,100
3,20,13,0.65,7,0.35,False,False,20,100,False,...,20,100,8,0.40,12,0.60,False,False,20,100
4,7,4,0.57,3,0.43,False,False,7,100,False,...,7,100,1,0.14,6,0.86,False,False,7,100
5,18,8,0.44,10,0.56,False,False,18,100,False,...,18,100,7,0.39,11,0.61,False,False,18,100
6,74,40,0.54,34,0.46,False,False,74,100,False,...,74,99,22,0.30,52,0.70,False,False,74,100
7,17,16,0.94,1,0.06,False,False,17,100,False,...,17,100,4,0.24,13,0.76,False,False,17,100
8,34,20,0.59,14,0.41,False,False,34,100,True,...,34,100,12,0.35,22,0.65,False,False,34,100
9,324,189,0.58,135,0.42,False,False,324,100,False,...,324,100,112,0.35,212,0.65,False,False,324,100


In [4]:
#Frequency Based Encoding
def frequency_encoding(data):
  encoder=['00','01','11','10']
  details= dict()
  for j in data.columns:
    col=data[j]
    sortx = dict()
    a=list()
    for i in col:
      if i not in sortx:
        sortx[i] = 1
        a.append(i)
      else:
        sortx[i]+=1
    a.sort()
    p=0
    s=0
    i=0
    l=0
    while p<len(data):
      m=int(sortx[a[l]])
      if  s+m <=(math.ceil(len(data)/len(encoder))) :
        s+=m
        sortx[a[l]]=encoder[i]
      else:
        if l == 0:
          sortx[a[l]]=encoder[i]
          i+=1
        else:
          i+=1
          sortx[a[l]]=encoder[i]
        s=0
      l+=1
      p+=m
    details[j]=sortx
  data1 = df(columns = data.columns)
  for i in range(len(data)):
    t=[]
    p=data.loc[i]
    for j in range(len(p)):
      s=details[data.columns[j]]
      t.append(s[p[j]])
    ar = pd.Series(t, index = data1.columns)
    data1 = data1.append(ar,ignore_index = True)
  return data1



In [5]:
#Generating final encoded string
def get_concatenated_string(data):
  enc=[]
  for i in range(len(data)):
    p=data.loc[i]
    t=""
    for j in p:
      t+=j
    enc.append(t)
  return enc

In [44]:
def split_string(encodings, split = 5):
  r_ind=[]
  enc_length = len(encodings[0])
  div = math.floor((enc_length)/split)+1
  print(div)
  for i in range(div, enc_length, div):
    r_ind.append(i)
  print(r_ind)
  t=[]
  for i in range(len(encodings)):
    s=0
    for j in r_ind:
      t.append(encodings[i][s:j])
      s=j
    t.append(encodings[i][s:])
    encodings[i]=t
    t=[]
  # for i in encodings:
    # print(i)
  return(encodings)  



In [45]:
#Function to add null boundary and returns string
def nullbound(n,winsize,t1):
  if n%2!=0:
          if winsize%2!=0:
            t1 = "0"*(winsize//2) + t1 + "0"*(winsize//2)
          else:
            t1 = "0"*((winsize//2)+1) + t1 + "0"*(winsize//2)
  else:
          if winsize%2!=0:
            t1 = "0"*(winsize//2) + t1 + "0"*(winsize//2)
          else:
            t1 = "0"*((winsize//2)+1) + t1 + "0"*(winsize//2)
  return t1

In [46]:
#generating binary rule
def rg(rule, winsize):
  brule=bin(rule).replace("0b","").zfill(2**winsize)
  brule=brule[::-1]
  return brule 

In [47]:
def get_rule_list(path="../rule_list.txt"):
    # opening the file in read mode
    my_file = open(path, "r")
    # reading the file
    file_con = my_file.read()
    # replacing end splitting the text 
    # when newline ('\n') is seen.
    rule_list = file_con.split("\n")
    my_file.close()
    return rule_list

In [48]:
#For finding the length of the next encoding(as in 2-bit, 3-bit encoding)
def findn(d):
    i=1
    while 2**i<d:
      i+=1
    return i

def cy_enc(l,index):
    return bin(index).replace("0b","").zfill(l)

In [49]:
#Will apply the CA rule and generate all possible cycles and returns nested list
def apply_rule(a1,winsize,brule):
  final_array = []
  a=list(set(a1))
  n=len(a[0])
  current_array=[]
  while(a):
    t1=a[0]
    flag=0
    while(not flag):
      if current_array == []:
        current_array.append(t1)
        a.remove(t1)
        
      else:
        t1=nullbound(n,winsize,t1)
        t2=""

        for j in range(n):
          check=int(str(t1[j:j+winsize]),2)
          t2+=brule[check]

        t1=t2

        if t1 not in current_array:
            if t1 in a:
              a.remove(t1)
              current_array.append(t1)
              
        else:
            flag=1
        

    final_array.append(current_array)
    current_array=[]
    cycle=1
    # print(cycle)
    cycle+=1
  return final_array

In [52]:
def cellular_automata_clustering(split, rule_list, encoding, window_size = 5):

  better_score_list = []
  best_CA_sill = -10000
  best_rule = []
  
  for rules in range(5):
    tmc=[]
    tma=[]
    print("Trial number :", rules)
    rule_set=random.sample(rule_list,2)
    #Clustering, checking scores 252702960
    #rule_set=[252691440,2218767375]
    # rule_set=[[1511938590,2218767375],[2276755335,1259293455],[2276755215,255652080]]
    #rule_set=[[1511938590,2218767375]]
    # rule_set=[267390795,267416079]
    # rule_set=[256577355,259190799]
    print(rule_set)
    enc1 = copy.deepcopy(encoding)
    for p in range(int(len(rule_set)/2)):
      rule = rg(int(rule_set[0]), window_size)
      fc = {}
      tr = []
      for i in range(split):
        for j in range(len(enc1)):
          tr.append(enc1[j][i])
        fc[i]=apply_rule(tr,window_size,rule)
        tr = []

      for i in range(split):
        t=fc[i]
        s=findn(len(t))
        for j in range(len(enc1)):
          plt=enc1[j][i]
          l=0
          while(plt not in t[l]):
            l+=1
          enc1[j][i]= cy_enc(s,l)
      rule=rg(int(rule_set[1]),window_size)
      for i in range(len(enc1)):
        t=""
        for j in enc1[i]:
          t+=j
        enc1[i]=t
      # print(len(enc1))
      fa=apply_rule(enc1,window_size,rule)
      #print(fa)
      kl=copy.deepcopy(enc1)
      for i in range(len(enc1)):
        j=0
        while j <len(fa):
          if enc1[i] in fa[j]:
            enc1[i]=j
            break
          else:
            j+=1

      
      X=data.to_numpy()
      #print(len(enc1),len(X))
      print(p)
      
      if (len(set(enc1))>1):
        CA_sill_old=silhouette_score(X,enc1,metric="euclidean")
        print(CA_sill_old)
        # print(rule_set[2*p+1])
        print("silhoutte:",CA_sill_old)
        # !!! CHange this later
        # if CA_sill_old < best_CA_old:
        #   best_CA_old = CA_sill_old
        #   best_rules= rule_set
        # print("calinski:",calinski_harabasz_score(X,enc1))
        # print("davies:",davies_bouldin_score(X,enc1))

        

        clust=KMeans(n_clusters=len(set(enc1)),random_state=42)
        clust.fit(X)
        labels=clust.labels_
        
        Kmean_sill_old=metrics.silhouette_score(X, labels, metric='euclidean')
        # print("Kmeans silhoutte :" ,Kmeans_sill_new)
        # print(len(set(labels)))
        # print("Kmeans calinski :", (metrics.calinski_harabasz_score(X, labels)))
        # print("Kmeans davies_bouldin_score: ",metrics.davies_bouldin_score(X, labels))
        print("-----------")
        # if Kmean_sill_old < CA_sill_old:
        #   better_score_list.append([rule_set, CA_sill_old, Kmean_sill_old])
        tmc.append([CA_sill_old,Kmean_sill_old])
        if tma==[] or tma[0][0]<CA_sill_old:
          tma.append([CA_sill_old,Kmean_sill_old])

    kls=copy.deepcopy(kl)
    t1=copy.deepcopy(fa) 
    for i in range(len(fa)-2):
      #print(len(t1),"###")
      kl=copy.deepcopy(kls)  
      fa1=copy.deepcopy(t1)
      idx=0
      for j in range(len(fa1)):
        if len(fa1[j])<len(fa1[idx]):
          idx=j
          
      t=copy.deepcopy(t1) 
      removed=t.pop(idx)
      #print(removed)
      ml=[]

      for j in removed:
        ml.append([kls.index(j),j])
      ml=sorted(ml)
      for j in ml:
        kl.remove(j[1])
      #print(ml)
      for j in range(len(fa1[idx])):
        p=[]
        a,b=ml[0]
        kl.insert(a,b)
        X=data.copy(deep=True)
        if len(ml)>1:
            for h in ml[1:]:
              X.drop(h[0],inplace=True)
          #print(len(X))
        X=X.to_numpy()
        for k in range(len(t)):
          m=copy.deepcopy(t)
          m[k].append(b)

          kle=copy.deepcopy(kl)
          for z in range(len(kle)):
            c=0
            while c <len(m):
              if kle[z] in m[c]:
                kle[z]=c
                break
              else:
                c+=1
          CA_sill_new=silhouette_score(X,kle,metric="euclidean")
          p.append(CA_sill_new)
        #print(p.index(max(p)))
        #print(t[p.index(max(p))])
        t[p.index(max(p))].append(fa1[idx][j])
        ml.pop()
        t1=t
        l=[]
        for i in t1:
          l.append(len(i))
        #print(l)
    for i in range(len(kls)):
      j=0
      while j <len(t1):
        if kls[i] in t1[j]:
          kls[i]=j
          break
        else:
          j+=1
    X=data.to_numpy()
    CA_sill_new=silhouette_score(X,kls,metric="euclidean")
    # print(best_rules)
    print("silhoutte:",CA_sill_new)
    print("calinski:",calinski_harabasz_score(X,kls))
    print("davies:",davies_bouldin_score(X,kls))
    clust=KMeans(n_clusters=len(set(kls)),random_state=42)
    clust.fit(X)
    labels=clust.labels_
        
    Kmeans_sill_new=metrics.silhouette_score(X, labels, metric='euclidean')


    if CA_sill_new > Kmeans_sill_new:
      better_score_list.append(rule_set)
    
    if CA_sill_new > best_CA_sill:
      best_CA_sill = CA_sill_new
      best_rule = rule_set


    my_data=[[len(fa),tma[0][0],tma[0][1],CA_sill_new,Kmeans_sill_new]]

    head=["Initial no.of clusters", "Silhoutte score for our algo","silhoutte score for kmeans"," new silhoutte score","new silhoute for kmeans"]
    print(tabulate(my_data, headers=head, tablefmt="grid"))
    p=[]
    for i in fa:
      p.append(len(i))
    q=[]
    for i in t1:
      q.append(len(i))
    print("before :",p)
    print("after: ",q)

    print(t1)
  return best_rule, best_CA_sill, better_score_list

In [53]:
#Replacing the data in table with encoding
freq_data = frequency_encoding(data)
enc = get_concatenated_string(freq_data)
split_enc = split_string(enc, split=5)
rule_list = get_rule_list()
best_rule, best_CA_sill, better_score_list = cellular_automata_clustering(5, rule_list, split_enc)

/var/folders/j7/0xc74mdd0vn8k3qvwmg5vl840000gn/T/ipykernel_15534/1616988999.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data1 = data1.append(ar,ignore_index = True)


18
[18, 36, 54, 72]
Trial number : 0
['3530698098', '4027544304']
0
0.06809686385135232
silhoutte: 0.06809686385135232
-----------
silhoutte: 0.09827129469555065
calinski: 4.679281083192244
davies: 2.1117809867870445
+--------------------------+--------------------------------+------------------------------+------------------------+---------------------------+
|   Initial no.of clusters |   Silhoutte score for our algo |   silhoutte score for kmeans |    new silhoutte score |   new silhoute for kmeans |
+==========================+================================+==============================+========================+===========================+
|                       26 |                      0.0680969 |                     0.203847 |              0.0982713 |                  0.766361 |
+--------------------------+--------------------------------+------------------------------+------------------------+---------------------------+
before : [1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


0
0.05128192028547315
silhoutte: 0.05128192028547315
-----------
silhoutte: 0.35436175695884603
calinski: 6.646506479454392
davies: 1.5614258590471501
+--------------------------+--------------------------------+------------------------------+------------------------+---------------------------+
|   Initial no.of clusters |   Silhoutte score for our algo |   silhoutte score for kmeans |    new silhoutte score |   new silhoute for kmeans |
+==========================+================================+==============================+========================+===========================+
|                        8 |                      0.0512819 |                     0.540252 |               0.354362 |                  0.766361 |
+--------------------------+--------------------------------+------------------------------+------------------------+---------------------------+
before : [17, 1, 4, 2, 1, 1, 1, 1]
after:  [21, 7]
[['1001101011100101000010001', '0011000011011000100001101', '0111000

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


0
-0.47985379641414305
silhoutte: -0.47985379641414305
-----------
silhoutte: 0.031278435822592994
calinski: 4.168714359241607
davies: 2.0284637079407593
+--------------------------+--------------------------------+------------------------------+------------------------+---------------------------+
|   Initial no.of clusters |   Silhoutte score for our algo |   silhoutte score for kmeans |    new silhoutte score |   new silhoute for kmeans |
+==========================+================================+==============================+========================+===========================+
|                        5 |                      -0.479854 |                     0.634825 |              0.0312784 |                  0.766361 |
+--------------------------+--------------------------------+------------------------------+------------------------+---------------------------+
before : [17, 8, 1, 1, 1]
after:  [17, 11]
[['010100101001011001010010', '011011001000111010011000', '01011000010001

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to '

0
-0.33215410048991373
silhoutte: -0.33215410048991373
-----------
silhoutte: 0.32220313663643413
calinski: 2.934030850210766
davies: 1.9576781579397668
+--------------------------+--------------------------------+------------------------------+------------------------+---------------------------+
|   Initial no.of clusters |   Silhoutte score for our algo |   silhoutte score for kmeans |    new silhoutte score |   new silhoute for kmeans |
+==========================+================================+==============================+========================+===========================+
|                        3 |                      -0.332154 |                     0.520635 |               0.322203 |                  0.766361 |
+--------------------------+--------------------------------+------------------------------+------------------------+---------------------------+
before : [3, 4, 12]
after:  [4, 15]
[['00001000101', '01101100101', '00000000000', '00000001100'], ['01001110001', '0

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [35]:
better_score_list

[]

before : [1, 1, 1, 2, 1, 1]
after:  [5, 2]
[['0000001', '0000101', '0100000', '0000000', '0000010'], ['0001000', '0000100']]


In [25]:
from sklearn.cluster import KMeans, MeanShift, DBSCAN, Birch
from sklearn import datasets
X=data.to_numpy()


In [26]:
clust_model = MeanShift(bandwidth=50)
clust_model.fit(X)
labels = clust_model.labels_
Kmeans_sill_new=metrics.silhouette_score(X, labels, metric='euclidean')
print("Meanshift silhoutte :" ,Kmeans_sill_new)
print("Meanshift calinski :", (metrics.calinski_harabasz_score(X, labels)))
print("Meanshift davies_bouldin_score: ",metrics.davies_bouldin_score(X, labels))


Meanshift silhoutte : 0.46821835837440073
Meanshift calinski : 350.2929624179578
Meanshift davies_bouldin_score:  0.23522706626769807


In [27]:
clust_model = DBSCAN(min_samples=2, eps=100)
clust_model.fit(X)
# Evaluating model's performance

labels = clust_model.labels_
# Kmeans_sill_new=metrics.silhouette_score(X, labels, metric='euclidean')
print("DBSCAN silhoutte :" ,Kmeans_sill_new)
# print("DBSCAN calinski :", (metrics.calinski_harabasz_score(X, labels)))
# print("DBSCAN davies_bouldin_score: ",metrics.davies_bouldin_score(X, labels))


DBSCAN silhoutte : 0.46821835837440073


In [28]:
clust_model = Birch(n_clusters=2,branching_factor=1000,threshold=1.5)
clust_model.fit(X)
labels = clust_model.labels_
Kmeans_sill_new=metrics.silhouette_score(X, labels, metric='euclidean')
print("Birch silhoutte :" ,Kmeans_sill_new)
print("Birch calinski :", (metrics.calinski_harabasz_score(X, labels)))
print("Birch davies_bouldin_score: ",metrics.davies_bouldin_score(X, labels))


Birch silhoutte : 0.7663611080988648
Birch calinski : 94.42893185124181
Birch davies_bouldin_score:  0.43429835583227294


In [29]:
X


array([[11, 4, 0.36, ..., False, 11, 100],
       [124, 70, 0.56, ..., False, 124, 100],
       [38, 26, 0.68, ..., False, 38, 100],
       ...,
       [6, 4, 0.67, ..., False, 6, 100],
       [0, 0, 0.0, ..., False, 0, 0],
       [0, 0, 0.0, ..., False, 0, 0]], dtype=object)

In [30]:
#Clustering, checking scores 252702960
winsize = 5
split=5

#rule_list=[267390795,267416079]
#rule_list=[256577355,259190799]
#rule_list=[252691440,2218767375]
# rule_list=[[1511938590,2218767375],[2276755335,1259293455],[2276755215,255652080]]
#rule_list=[[1511938590,2218767375]]

tmc=[]
tma=[]
enc1 = copy.deepcopy(enc)
for p in range(int(len(rule_list)/2)):
  rule = rg(int(rule_list[0]), window_size)
  fc = {}

  tr = []
  for i in range(split):
    for j in range(len(enc1)):
      tr.append(enc1[j][i])
    fc[i]=apply_rule(tr,winsize,rule)
    tr = []

  for i in range(split):
    t=fc[i]
    s=findn(len(t))
    for j in range(len(enc1)):
      plt=enc1[j][i]
      l=0
      while(plt not in t[l]):
        l+=1
      enc1[j][i]= cy_enc(s,l)
  rule=rg(int(rule_list[1]), window_size)
  for i in range(len(enc1)):
    t=""
    for j in enc1[i]:
      t+=j
    enc1[i]=t
  # print(len(enc1))
  fa=apply_rule(enc1,winsize,rule)
  #print(fa)
  kl=copy.deepcopy(enc1)
  for i in range(len(enc1)):
    j=0
    while j <len(fa):
      if enc1[i] in fa[j]:
        enc1[i]=j
        break
      else:
        j+=1

  
  X=data.to_numpy()
  #print(len(enc1),len(X))
  print(p)
  
  if (len(set(enc1))>1):
      CA_sill_new=silhouette_score(X,enc1,metric="euclidean")
      # print(rule_list[2*p])
      # print(rule_list[2*p+1])
      # print("silhoutte:",CA_sill_new)
      # print("calinski:",calinski_harabasz_score(X,enc1))
      # print("davies:",davies_bouldin_score(X,enc1))

      clust=KMeans(n_clusters=len(set(enc1)),random_state=42)
      clust.fit(X)
      labels=clust.labels_
      
      Kmeans_sill_new=metrics.silhouette_score(X, labels, metric='euclidean')
      # print("Kmeans silhoutte :" ,Kmeans_sill_new)
      print(len(set(labels)))
      # print("Kmeans calinski :", (metrics.calinski_harabasz_score(X, labels)))
      # print("Kmeans davies_bouldin_score: ",metrics.davies_bouldin_score(X, labels))
      print("-----------")
      tmc.append([CA_sill_new,Kmeans_sill_new])
      if tma==[] or tma[0][0]<CA_sill_new:
        tma=[[CA_sill_new,Kmeans_sill_new]]
  #enc1=copy.deepcopy(enc)




  



0
6
-----------


In [31]:
kls=copy.deepcopy(kl)
t1=copy.deepcopy(fa) 
for i in range(len(fa)-2):
  #print(len(t1),"###")
  kl=copy.deepcopy(kls)  
  fa1=copy.deepcopy(t1)
  idx=0
  for j in range(len(fa1)):
    if len(fa1[j])<len(fa1[idx]):
      idx=j
      
  t=copy.deepcopy(t1) 
  removed=t.pop(idx)
  #print(removed)
  ml=[]

  for j in removed:
    ml.append([kls.index(j),j])
  ml=sorted(ml)
  for j in ml:
    kl.remove(j[1])
  #print(ml)
  for j in range(len(fa1[idx])):
    p=[]
    a,b=ml[0]
    kl.insert(a,b)
    X=data.copy(deep=True)
    if len(ml)>1:
        for h in ml[1:]:
          X.drop(h[0],inplace=True)
      #print(len(X))
    X=X.to_numpy()
    for k in range(len(t)):
      m=copy.deepcopy(t)
      m[k].append(b)

      kle=copy.deepcopy(kl)
      for z in range(len(kle)):
        c=0
        while c <len(m):
          if kle[z] in m[c]:
            kle[z]=c
            break
          else:
            c+=1
      CA_sill_new=silhouette_score(X,kle,metric="euclidean")
      p.append(CA_sill_new)
    #print(p.index(max(p)))
    #print(t[p.index(max(p))])
    t[p.index(max(p))].append(fa1[idx][j])
    ml.pop()
    t1=t
    l=[]
    for i in t1:
      l.append(len(i))
    #print(l)

In [32]:
len(kl)

32

In [33]:
for i in range(len(kls)):
    j=0
    while j <len(t1):
      if kls[i] in t1[j]:
        kls[i]=j
        break
      else:
        j+=1
X=data.to_numpy()
CA_sill_new=silhouette_score(X,kls,metric="euclidean")
print("silhoutte:",CA_sill_new)
print("calinski:",calinski_harabasz_score(X,kls))
print("davies:",davies_bouldin_score(X,kls))
clust=KMeans(n_clusters=len(set(kls)),random_state=42)
clust.fit(X)
labels=clust.labels_
      
Kmeans_sill_new=metrics.silhouette_score(X, labels, metric='euclidean')
print("Kmeans silhoutte :" ,Kmeans_sill_new)
print("Kmeans calinski :", (metrics.calinski_harabasz_score(X, labels)))
print("Kmeans davies_bouldin_score: ",metrics.davies_bouldin_score(X, labels))
print(len(set(labels)))

silhoutte: 0.443505982241705
calinski: 4.60603342090343
davies: 1.3405901839679435
Kmeans silhoutte : 0.7663611080988648
Kmeans calinski : 94.42893185124181
Kmeans davies_bouldin_score:  0.43429835583227294
2


In [34]:
my_data=[[len(fa),tma[0][0],tma[0][1],CA_sill_new,Kmeans_sill_new]]
head=["Initial no.of clusters", "Silhoutte score for our algo","silhoutte score for kmeans"," new silhoutte score","new silhoute for kmeans"]
print(tabulate(my_data, headers=head, tablefmt="grid"))

+--------------------------+--------------------------------+------------------------------+------------------------+---------------------------+
|   Initial no.of clusters |   Silhoutte score for our algo |   silhoutte score for kmeans |    new silhoutte score |   new silhoute for kmeans |
+==========================+================================+==============================+========================+===========================+
|                        6 |                      -0.302546 |                     0.635624 |               0.443506 |                  0.766361 |
+--------------------------+--------------------------------+------------------------------+------------------------+---------------------------+


In [ ]:
p=[]
for i in fa:
  p.append(len(i))
q=[]
for i in t1:
  q.append(len(i))
print("before :",p)
print("after: ",q)

print(t1)

before : [25, 1, 1]
after:  [25, 2]
[['0100010100001000010', '0010001010000101010', '0000000000010000011', '1000000000000100010', '0001000010100001101', '0000000000010000001', '0001001001000001101', '0001100101100001000', '0100100000111000010', '0100010000000100010', '0100000001110110010', '0101101000000011001', '0010000000000010010', '0001000100000001101', '0100001100000100010', '0111011110010111110', '0011011000000011001', '0101010101110000001', '0000011100000000001', '0001001000010001000', '0110011000100000001', '0010000001000110010', '0101100000000011001', '0000100000000000001', '0010001000001000010'], ['0001000000000001101', '0001000001010001100']]


In [ ]:
from sklearn.cluster import KMeans, MeanShift, DBSCAN, Birch
from sklearn import datasets
X=data.to_numpy()


In [ ]:
clust_model = MeanShift(bandwidth=50)
clust_model.fit(X)
labels = clust_model.labels_
Kmeans_sill_new=metrics.silhouette_score(X, labels, metric='euclidean')
print("Meanshift silhoutte :" ,Kmeans_sill_new)
print("Meanshift calinski :", (metrics.calinski_harabasz_score(X, labels)))
print("Meanshift davies_bouldin_score: ",metrics.davies_bouldin_score(X, labels))


Meanshift silhoutte : 0.46821835837440073
Meanshift calinski : 350.2929624179578
Meanshift davies_bouldin_score:  0.23522706626769807


In [ ]:
clust_model = DBSCAN(min_samples=2, eps=100)
clust_model.fit(X)
# Evaluating model's performance

labels = clust_model.labels_
# Kmeans_sill_new=metrics.silhouette_score(X, labels, metric='euclidean')
print("DBSCAN silhoutte :" ,Kmeans_sill_new)
# print("DBSCAN calinski :", (metrics.calinski_harabasz_score(X, labels)))
# print("DBSCAN davies_bouldin_score: ",metrics.davies_bouldin_score(X, labels))


DBSCAN silhoutte : 0.46821835837440073


In [ ]:
clust_model = Birch(n_clusters=2,branching_factor=1000,threshold=1.5)
clust_model.fit(X)
labels = clust_model.labels_
Kmeans_sill_new=metrics.silhouette_score(X, labels, metric='euclidean')
print("Birch silhoutte :" ,Kmeans_sill_new)
print("Birch calinski :", (metrics.calinski_harabasz_score(X, labels)))
print("Birch davies_bouldin_score: ",metrics.davies_bouldin_score(X, labels))


Birch silhoutte : 0.7663611080988648
Birch calinski : 94.42893185124181
Birch davies_bouldin_score:  0.43429835583227294


In [ ]:
X


array([[ 11.  ,   4.  ,   0.36, ...,   0.  ,  11.  , 100.  ],
       [124.  ,  70.  ,   0.56, ...,   0.  , 124.  , 100.  ],
       [ 38.  ,  26.  ,   0.68, ...,   0.  ,  38.  , 100.  ],
       ...,
       [  6.  ,   4.  ,   0.67, ...,   0.  ,   6.  , 100.  ],
       [  0.  ,   0.  ,   0.  , ...,   0.  ,   0.  ,   0.  ],
       [  0.  ,   0.  ,   0.  , ...,   0.  ,   0.  ,   0.  ]])

In [19]:
temp = {}


In [21]:
temp['2'] = 4

In [22]:
temp

{2: 4, '2': 4}